# 사용한 라이브러리

In [12]:
import random
import numpy as np
from collections import deque
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomUniform
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import time
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm.notebook import tqdm as tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

# NN

In [13]:
class NN(tf.keras.Model):
    def __init__(self, action_size):
        super(NN, self).__init__()
        self.fc1 = Dense(24, activation = 'relu')
        self.fc2 = Dense(24, activation = 'relu')
        self.fc_out = Dense(action_size, kernel_initializer = RandomUniform(-1e-3, 1e-3))

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        q = self.fc_out(x)
        return q

# DQN

In [14]:
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size

        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.001
        self.batch_size = 64
        self.train_start = 1000

        self.memory = deque(maxlen = 2000)

        self.model = NN(self.action_size)
        self.target_model = NN(self.action_size)
        self.optimizer = Adam(learning_rate = self.learning_rate)

        self.update_target_model()

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            q = self.model(state) # 리스트 형태로 반환됨
            return np.argmax(q[0])

    def append_sample(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train_model(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
        mini_batch = random.sample(self.memory, self.batch_size)
        states = np.array([sample[0][0] for sample in mini_batch])
        actions = np.array([sample[1] for sample in mini_batch])
        rewards = np.array([sample[2] for sample in mini_batch])
        next_states = np.array([sample[3][0] for sample in mini_batch])
        dones = np.array([sample[4] for sample in mini_batch])

        model_params = self.model.trainable_variables
            
        with tf.GradientTape() as tape:
            predicts = self.model(states)
            one_hot_action = tf.one_hot(actions , self.action_size)
            predicts = tf.reduce_sum(one_hot_action * predicts, axis=1)
        
            target_predicts = self.target_model(next_states)
            target_predicts = tf.stop_gradient(target_predicts)
        
            max_q = np.amax(target_predicts, axis=-1)
            targets = rewards + (1-dones) * self.discount_factor * max_q
            loss = tf.reduce_mean(tf.square(targets - predicts))
        
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))

# parameter 정의

In [15]:
state_size = 1
action_size = 3
index = 0 # GRU
agent = DQN(state_size, action_size)

scores, episodes = [], []
score_avg = 0

EPISODES = 20000

# GRU 모델 불러오기

In [16]:
# GRU 모델에서 window_size = 100인 경우, 중단기적인 예측 결과
gru_model_100 = gru_model_100.load_state_dict(torch.load("./models/gru_model_100.pt"))
gru_model_1 = gru_model_1.load_state_dict(torch.load("./models/gru_model_1.pt"))

AttributeError: 'numpy.ndarray' object has no attribute 'load_state_dict'

# EPISODE

In [ ]:
for e in range(EPISODES):
    done = False
    score = 0
    
    state = new_real[0] # 전력 수요량 데이터 한 점
    state = np.reshape(state, [1, state_size])
    
    # data_length = new_gru_output_1.shape
    count = 0
    index = 1
    shot = 0
    long =0
    while not done:
        action = agent.get_action(state) # 내가 행동을 결정하는 방법
        
        if action == 0:
            next_state = new_gru_output_1[index + 1]
            shot = next_state
        elif action == 1:
            next_state = new_gru_output_100[index + 1]
            long = next_state
            
        next_state = np.reshape(next_state, [1, state_size]) # next_state를 [1, state_size]의 배열로 재배열 
        # print("___________________________________")
        # print(f"GRU의 출력 데이터 : {state}")
            
        # reward 계산 > 이 부분은 후에 구체적으로 변경 다시 해야 함
        if next_state == shot:
            if abs(new_real[index + 1] - shot) < abs(new_real[index + 1] - long) : # (ra - 모델이 선택한 값) < (실제 값 - 선택 안된 값)
                reward = 1
            else:
                reward = -1
        elif next_state == long:
            if abs(new_real[index + 1] - long) < abs(new_real[index + 1] - shot) : # (ra - 모델이 선택한 값) < (실제 값 - 선택 안된 값)
                reward = 1
            else:
                reward = -1
        
        if count == '내가 예측할 데이터 수':
            done = True
            
        if reward < 0: # 막 정한 수
            done = True  
        #print('보상 : ', reward)
        index = index + 1
        score += reward
        agent.append_sample(state, action, reward, next_state, done)

        if len(agent.memory) >= agent.train_start:
            agent.train_model()

        
        # 다음 state 인덱스로 이동
        if index == 7803 - 1:
            done = True  # 데이터의 끝에 도달
        
        state = new_real[index]
        state = np.reshape(state, [1, state_size])
        
        if done:
            agent.update_target_model()
            count = count + 1
            print(f"episode : {e} | score : {score}")
            print("___________________________________")
            scores.append(score_avg)
            episodes.append(e)
       
    
    
import time
import datetime # datetime 라이브러리 import
 
start = time.time() # 시작
time.sleep(1) # 수행시간 측정하고자 하는 코드 부분
sec = time.time()-start # 종료 - 시작 (걸린 시간)
 
times = str(datetime.timedelta(seconds=sec)) # 걸린시간 보기좋게 바꾸기
short = times.split(".")[0] # 초 단위 까지만
print(f"{times} sec")
print(f"{short} sec")

episode : 0 | score : 7
___________________________________
episode : 1 | score : 2
___________________________________
episode : 2 | score : -1
___________________________________
episode : 3 | score : -1
___________________________________
episode : 4 | score : 12
___________________________________
episode : 5 | score : 0
___________________________________
episode : 6 | score : -1
___________________________________
episode : 7 | score : 3
___________________________________
episode : 8 | score : 5
___________________________________
episode : 9 | score : -1
___________________________________
episode : 10 | score : 10
___________________________________
episode : 11 | score : 0
___________________________________
episode : 12 | score : 3
___________________________________
episode : 13 | score : 8
___________________________________
episode : 14 | score : 0
___________________________________
episode : 15 | score : 1
___________________________________
episode : 16 | score : -1
__

episode : 240 | score : 7
___________________________________
episode : 241 | score : 6
___________________________________
episode : 242 | score : -1
___________________________________
episode : 243 | score : 22
___________________________________
episode : 244 | score : -1
___________________________________
episode : 245 | score : 7
___________________________________
episode : 246 | score : 5
___________________________________
episode : 247 | score : 1
___________________________________
episode : 248 | score : -1
___________________________________
episode : 249 | score : 0
___________________________________
episode : 250 | score : 6
___________________________________
episode : 251 | score : 3
___________________________________
episode : 252 | score : 1
___________________________________
episode : 253 | score : -1
___________________________________
episode : 254 | score : -1
___________________________________
episode : 255 | score : -1
___________________________________
e

episode : 372 | score : 3
___________________________________
episode : 373 | score : 0
___________________________________
episode : 374 | score : 3
___________________________________
episode : 375 | score : 8
___________________________________
episode : 376 | score : -1
___________________________________
episode : 377 | score : 4
___________________________________
episode : 378 | score : -1
___________________________________
episode : 379 | score : -1
___________________________________
episode : 380 | score : 21
___________________________________
episode : 381 | score : 0
___________________________________
episode : 382 | score : 12
___________________________________
episode : 383 | score : -1
___________________________________
episode : 384 | score : 3
___________________________________
episode : 385 | score : 2
___________________________________
episode : 386 | score : -1
___________________________________
episode : 387 | score : -1
___________________________________


episode : 506 | score : 26
___________________________________
episode : 507 | score : 1
___________________________________
episode : 508 | score : 22
___________________________________
episode : 509 | score : 27
___________________________________
episode : 510 | score : 0
___________________________________
episode : 511 | score : 86
___________________________________
episode : 512 | score : 21
___________________________________
episode : 513 | score : 4
___________________________________
episode : 514 | score : 21
___________________________________
episode : 515 | score : 43
___________________________________
episode : 516 | score : -1
___________________________________
episode : 517 | score : 12
___________________________________
episode : 518 | score : 9
___________________________________
episode : 519 | score : 0
___________________________________
episode : 520 | score : 50
___________________________________
episode : 521 | score : 101
________________________________

episode : 636 | score : 849
___________________________________
episode : 637 | score : 17
___________________________________
episode : 638 | score : 51
___________________________________
episode : 639 | score : -1
___________________________________
episode : 640 | score : -1
___________________________________
episode : 641 | score : -1
___________________________________
episode : 642 | score : -1
___________________________________
episode : 643 | score : -1
___________________________________
episode : 644 | score : 16
___________________________________
episode : 645 | score : 2
___________________________________
episode : 646 | score : -1
___________________________________
episode : 647 | score : -1
___________________________________
episode : 648 | score : -1
___________________________________
episode : 649 | score : -1
___________________________________
episode : 650 | score : -1
___________________________________
episode : 651 | score : -1
____________________________

...

episode : 6985 | score : -1
___________________________________
episode : 6986 | score : -1
___________________________________
episode : 6987 | score : -1
___________________________________
episode : 6988 | score : -1
___________________________________
episode : 6989 | score : -1
___________________________________
episode : 6990 | score : -1
___________________________________
episode : 6991 | score : -1
___________________________________
episode : 6992 | score : -1
___________________________________
episode : 6993 | score : -1
___________________________________
episode : 6994 | score : -1
___________________________________
episode : 6995 | score : -1
___________________________________
episode : 6996 | score : -1
___________________________________
episode : 6997 | score : -1
___________________________________
episode : 6998 | score : -1
___________________________________
episode : 6999 | score : -1
___________________________________
episode : 7000 | score : -1
____________

episode : 7119 | score : -1
___________________________________
episode : 7120 | score : -1
___________________________________
episode : 7121 | score : -1
___________________________________
episode : 7122 | score : -1
___________________________________
episode : 7123 | score : -1
___________________________________
episode : 7124 | score : -1
___________________________________
episode : 7125 | score : -1
___________________________________
episode : 7126 | score : -1
___________________________________
episode : 7127 | score : -1
___________________________________
episode : 7128 | score : -1
___________________________________
episode : 7129 | score : -1
___________________________________
episode : 7130 | score : -1
___________________________________
episode : 7131 | score : -1
___________________________________
episode : 7132 | score : -1
___________________________________
episode : 7133 | score : -1
___________________________________
episode : 7134 | score : -1
____________

episode : 7247 | score : 1941
___________________________________
episode : 7248 | score : 426
___________________________________
episode : 7249 | score : 2552
___________________________________
episode : 7250 | score : 7801
___________________________________
episode : 7251 | score : 4861
___________________________________
episode : 7252 | score : 5604
___________________________________
episode : 7253 | score : 1255
___________________________________
episode : 7254 | score : 6139
___________________________________
episode : 7255 | score : -1
___________________________________
episode : 7256 | score : -1
___________________________________
episode : 7257 | score : -1
___________________________________
episode : 7258 | score : -1
___________________________________
episode : 7259 | score : -1
___________________________________
episode : 7260 | score : -1
___________________________________
episode : 7261 | score : -1
___________________________________
episode : 7262 | score : 

episode : 7379 | score : 66
___________________________________
episode : 7380 | score : 4527
___________________________________
episode : 7381 | score : 356
___________________________________
episode : 7382 | score : 2347
___________________________________
episode : 7383 | score : 870
___________________________________
episode : 7384 | score : 6057
___________________________________
episode : 7385 | score : 7801
___________________________________
episode : 7386 | score : 3032
___________________________________
episode : 7387 | score : 872
___________________________________
episode : 7388 | score : 236
___________________________________
episode : 7389 | score : 7801
___________________________________
episode : 7390 | score : 4320
___________________________________
episode : 7391 | score : -1
___________________________________
episode : 7392 | score : -1
___________________________________
episode : 7393 | score : -1
___________________________________
episode : 7394 | score

episode : 7506 | score : 4138
___________________________________
episode : 7507 | score : 86
___________________________________
episode : 7508 | score : 856
___________________________________
episode : 7509 | score : 7801
___________________________________
episode : 7510 | score : 37
___________________________________
episode : 7511 | score : 3075
___________________________________
episode : 7512 | score : 1917
___________________________________
episode : 7513 | score : 7801
___________________________________
episode : 7514 | score : 2279
___________________________________
episode : 7515 | score : -1
___________________________________
episode : 7516 | score : -1
___________________________________
episode : 7517 | score : -1
___________________________________
episode : 7518 | score : -1
___________________________________
episode : 7519 | score : -1
___________________________________
episode : 7520 | score : -1
___________________________________
episode : 7521 | score : -1

In [ ]:
import matplotlib.pyplot as plt
plt.plot(episodes, scores)